# 1D Reactor

In [1]:
import sys
import numpy as np
import pandas as pd
import cantera as ct

In [ ]:
N_ex = 2000
N_samp = 50
dt = 5.0e-7

phi = 1.0
T_min = 1000.0
T_max = 3000.0
p = ct.one_atm

In [4]:
gas = ct.Solution('./ch41step.xml')
R = rxns = ct.Reaction.listFromFile('./ch41step.xml')
myreaction  = R[0]

iCO2 = gas.species_index('CO2')
iH2O = gas.species_index('H2O')
iO2 = gas.species_index('O2')
iCH4 = gas.species_index('CH4')

transient_data = pd.DataFrame()
constant_data = pd.DataFrame()

In [ ]:
for i in range(N_ex):
    T = T_min + (T_max-T_min)*np.random.rand()
    gas.TPY = T, p, 'CH4:{},O2:2'.format(phi)    
    time = 0.0
    
    r = ct.IdealGasConstPressureReactor(gas, energy = 'off')
    sim = ct.ReactorNet([r])
    
    states = ct.SolutionArray(gas, extra=['t'])
    transient_data = transient_data.append({'t [s]': sim.time, 'T [K]':  r.T
                                            ,'Yco2': r.Y[iCO2],'Yh2o': r.Y[iH2O],'Yo2':r.Y[iO2],'Ych4':r.Y[iCH4]},
                                            ignore_index=True)
    
    constant_data = constant_data.append({'T [K]':  r.T, 'P [Pa]': r.thermo.P, 'kf': gas.forward_rate_constants, 
                                          'kb':gas.reverse_rate_constants, 'Ea':myreaction.rate.activation_energy, 
                                          'A': myreaction.rate.pre_exponential_factor,
                                          'b': myreaction.rate.temperature_exponent},ignore_index=True)
    
    print("i = {}".format(i))
    
    for n in range(N_samp):
        time += dt
        sim.advance(time)
        states.append(r.thermo.state, t=time*1e3)
        transient_data = transient_data.append({'t [s]': sim.time, 'T [K]':  r.T
                                    ,'Yco2': r.Y[iCO2],'Yh2o': r.Y[iH2O],'Yo2':r.Y[iO2],'Ych4':r.Y[iCH4]},
                                               ignore_index=True)
        
transient_data.to_csv('transient_data_ch41step.csv') 
constant_data.to_csv('constant_data_ch41step.csv') 

# Check kf

In [5]:
phi = 1.0
T = 2000.0
p=1e5

gas.TPX = T, p, 'CH4:{},O2:2'.format(phi)

A = myreaction.rate.pre_exponential_factor
b = myreaction.rate.temperature_exponent
Ea = myreaction.rate.activation_energy

kf = A * T**b * np.exp(-Ea/(ct.gas_constant/4.184 * T))
print(kf)

0.24971793802434938


In [6]:
Ea

83680000.0

In [8]:
-Ea/(ct.gas_constant/4.184 * 300)

-140.36471062471577